In [1]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from sam2_setup import init_sam2
from functional_map import compute_surface_map

/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
device = torch.device('cuda:0')
torch.cuda.set_device(device)
num_views = 20
H = 512
W = 512
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [3]:
def compute_features(device, pipe, dino_model, m, prompt):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        pipe=pipe, 
        dino_model=dino_model,
        sam_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        num_images_per_prompt=num_images_per_prompt,
        use_normal_map=use_normal_map,
        use_sam=True,
        use_diffusion=False,
    )
    return features.cpu()

In [4]:
pipe = init_pipe(device)
dino_model = init_sam2(device)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [5]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [6]:
f_source = compute_features(device, pipe, dino_model, source_mesh, "cow")

Starting batch_render with num_views=20, H=512, W=512
Rendering completed successfully
Starting batch_render with num_views=20, H=512, W=512
Rendering completed successfully
batched_renderings shape:  torch.Size([40, 512, 512, 4])
normal_batched_renderings shape:  torch.Size([40, 512, 512, 1, 3])
camera R: tensor([[[-1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.0000e+00]],

        [[-9.5106e-01,  0.0000e+00, -3.0902e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 3.0902e-01,  0.0000e+00, -9.5106e-01]],

        [[-8.0902e-01,  0.0000e+00, -5.8779e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 5.8779e-01,  0.0000e+00, -8.0902e-01]],

        [[-5.8779e-01,  0.0000e+00, -8.0902e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 8.0902e-01,  0.0000e+00, -5.8779e-01]],

        [[-3.0902e-01,  0.0000e+00, -9.5106e-01],
         [ 0.0000e+00,  1.0000e+00

/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1729805341246/work/aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0%|                                                                                                                                                                                                                                     | 0/40 [00:00<?, ?it/s]

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/sam2/modeling/sam/transformer.py:270: UserWarning: Memory efficient kernel not used because: (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1729805341246/work/aten/src/ATen/native/transformers/cuda/sdp_utils.cpp:723.)
  out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_p)
/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/sam2/modeling/sam/transformer.py:270: UserWarning: Memory Efficient attention has been runtime disabled. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1729805341246/work/aten/src/ATen/native/transformers/sdp_utils_cpp.h:495.)
  out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_p)
/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/sam2/modeling/sam/transformer.py:270: UserWarning: Flash attention kernel not used because: (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


...

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


 98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 39/40 [02:46<00:04,  4.22s/it]

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [02:50<00:00,  4.26s/it]


Number of missing features:  6
Copied features from nearest vertices
Time taken in mins:  2.921879283587138


In [7]:
f_target = compute_features(device, pipe, dino_model, target_mesh, "camel")

Starting batch_render with num_views=20, H=512, W=512
Rendering completed successfully
Starting batch_render with num_views=20, H=512, W=512
Rendering completed successfully
batched_renderings shape:  torch.Size([40, 512, 512, 4])
normal_batched_renderings shape:  torch.Size([40, 512, 512, 1, 3])
camera R: tensor([[[-1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.0000e+00]],

        [[-9.5106e-01,  0.0000e+00, -3.0902e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 3.0902e-01,  0.0000e+00, -9.5106e-01]],

        [[-8.0902e-01,  0.0000e+00, -5.8779e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 5.8779e-01,  0.0000e+00, -8.0902e-01]],

        [[-5.8779e-01,  0.0000e+00, -8.0902e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 8.0902e-01,  0.0000e+00, -5.8779e-01]],

        [[-3.0902e-01,  0.0000e+00, -9.5106e-01],
         [ 0.0000e+00,  1.0000e+00

  0%|                                                                                                                                                                                                                                     | 0/40 [00:00<?, ?it/s]

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


  2%|█████▌                                                                                                                                                                                                                       | 1/40 [00:04<03:06,  4.79s/it]

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


...

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


 98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 39/40 [02:45<00:04,  4.29s/it]

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [02:50<00:00,  4.26s/it]

Number of missing features:  1
Copied features from nearest vertices
Time taken in mins:  2.9083128611246747


In [8]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]

In [9]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Apply functional map on features

In [10]:
source_file_path = "meshes/cat.off"
target_file_path = "meshes/lion.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "cat")
f_target = compute_features(device, pipe, dino_model, target_mesh, "lion")

Starting batch_render with num_views=20, H=512, W=512
Rendering completed successfully
Starting batch_render with num_views=20, H=512, W=512
Rendering completed successfully
batched_renderings shape:  torch.Size([40, 512, 512, 4])
normal_batched_renderings shape:  torch.Size([40, 512, 512, 1, 3])
camera R: tensor([[[-1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.0000e+00]],

        [[-9.5106e-01,  0.0000e+00, -3.0902e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 3.0902e-01,  0.0000e+00, -9.5106e-01]],

        [[-8.0902e-01,  0.0000e+00, -5.8779e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 5.8779e-01,  0.0000e+00, -8.0902e-01]],

        [[-5.8779e-01,  0.0000e+00, -8.0902e-01],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 8.0902e-01,  0.0000e+00, -5.8779e-01]],

        [[-3.0902e-01,  0.0000e+00, -9.5106e-01],
         [ 0.0000e+00,  1.0000e+00

  0%|                                                                                                                                                                                                                                     | 0/40 [00:00<?, ?it/s]

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


  2%|█████▌                                                                                                                                                                                                                       | 1/40 [00:05<03:44,  5.74s/it]

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


...

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


 98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 39/40 [02:53<00:04,  4.39s/it]

Type: <class 'list'>
Length: 2
First element type: <class 'torch.Tensor'>
Second element type: <class 'list'>


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [02:58<00:00,  4.45s/it]

Number of missing features:  0
Copied features from nearest vertices
Time taken in mins:  3.096793059508006


In [11]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [12]:
surface_map = compute_surface_map(source_file_path, target_file_path, f_source.numpy(), f_target.numpy())

mesh1 (7207, 3)
mesh2 (5000, 3)

Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 10.26 s
Computing 200 eigenvectors
	Done in 6.86 s

Computing descriptors
	Normalizing descriptors

	256 out of 2048 possible descriptors kept
Computing commutativity operators
	Scaling LBO commutativity weight by 8.5e-10

Optimization :
	50 Ev on source - 50 Ev on Target
	Using 256 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 56, nit : 51, warnflag : 0
	Done in 9.70 seconds


In [13]:
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[surface_map.cpu().numpy()]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Part segmentation

In [14]:
from sklearn.cluster import KMeans
import numpy as np

k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner
segments2 = kmeans.predict(f_target)

In [15]:
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [16]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [17]:
source_file_path = "meshes/posed_human.off"
target_file_path = "meshes/cat.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "naked human")
f_target = compute_features(device, pipe, dino_model, target_mesh, "cat")

Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:24<00:00,  2.04s/it]


Number of missing features:  3
Copied features from nearest vertices
Time taken in mins:  3.526184153556824
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:25<00:00,  2.06s/it]

Number of missing features:  140
Copied features from nearest vertices
Time taken in mins:  3.5948370456695558


In [18]:
k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner

segments2 = kmeans.predict(f_target)
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [19]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  